In [1]:
from rich import print
from pathlib import Path

In [2]:
from ultralytics.models.yolo.detect.val import DetectionValidator

In [3]:
args = dict(
    model = "/icislab/volume3/benderick/futurama/YOLO-UAV/logs/YOLO-UAV/runs/2025-05-24_11-03-11-MBFD/YOLO-UAV/MBFD/weights/best.pt",
    data = "/icislab/volume3/benderick/futurama/YOLO-UAV/data/VisDrone/VisDrone.yaml",
    conf=0.001,
    batch=16,
    device="cuda:1",
    save_json=True,
    split='test'
    )

save_dir = Path("./logs/val")

In [4]:
validator = DetectionValidator(save_dir=save_dir, args=args)

In [5]:
validator()

Ultralytics 8.3.88 🚀 Python-3.10.16 torch-2.6.0+cu118 CUDA:1 (NVIDIA GeForce RTX 3090, 24260MiB)
MBFD summary: 169 layers, 2,733,694 parameters, 0 gradients, 7.2 GFLOPs


🔔 val: Scanning /icislab/volume3/benderick/futurama/data/VisDrone/VisDrone2019-DET-test-dev/labels.cache... 1610 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1610/1610 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   2%|▏         | 2/101 [00:01<01:41,  1.03s/it]

WARNING ⚠️ NMS time limit 2.800s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   3%|▎         | 3/101 [00:06<03:59,  2.44s/it]

WARNING ⚠️ NMS time limit 2.800s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 101/101 [00:44<00:00,  2.26it/s]


                   all       1610      75102      0.338      0.258      0.233      0.127
            pedestrian       1197      21006      0.354      0.191      0.177     0.0665
                people        797       6376       0.35     0.0874      0.102     0.0327
               bicycle        377       1302      0.233     0.0238     0.0501     0.0192
                   car       1530      28074      0.531      0.674      0.637      0.384
                   van       1168       5771       0.28      0.379      0.283      0.176
                 truck        750       2659      0.335      0.289      0.245      0.142
              tricycle        245        530      0.223      0.132      0.103     0.0538
       awning-tricycle        233        599      0.312     0.0935      0.107     0.0561
                   bus        838       2940       0.43       0.45      0.423      0.267
                 motor        794       5845      0.331      0.264      0.204     0.0768
Speed: 0.3ms preproce

{'metrics/precision(B)': np.float64(0.3380169706160433),
 'metrics/recall(B)': np.float64(0.25845219751875137),
 'metrics/mAP50(B)': np.float64(0.23300004672371316),
 'metrics/mAP50-95(B)': np.float64(0.127458908931889),
 'fitness': np.float64(0.1380130227110714)}

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os
import numpy as np
from prettytable import PrettyTable
from ultralytics import YOLO
from ultralytics.utils.torch_utils import model_info

# BILIBILI UP 魔傀面具
# 验证参数官方详解链接：https://docs.ultralytics.com/modes/val/#usage-examples:~:text=of%20each%20category-,Arguments%20for%20YOLO%20Model%20Validation,-When%20validating%20YOLO

# 精度小数点保留位数修改问题可看<使用说明.md>下方的<YOLOV8源码常见疑问解答小课堂>第五点
# 最终论文的参数量和计算量统一以这个脚本运行出来的为准

def get_weight_size(path):
    stats = os.stat(path)
    return f'{stats.st_size / 1024 / 1024:.1f}'

if __name__ == '__main__':
    model_path = 'runs/train/exp/weights/best.pt'
    model = YOLO(model_path) # 选择训练好的权重路径
    result = model.val(data='/root/dataset/dataset_visdrone/data.yaml',
                        split='val', # split可以选择train、val、test 根据自己的数据集情况来选择.
                        imgsz=640,
                        batch=16,
                        # iou=0.7,
                        # rect=False,
                        # save_json=True, # if you need to cal coco metrice
                        project='runs/val',
                        name='exp',
                        )
    
    if model.task == 'detect': # 仅目标检测任务适用
        model_names = list(result.names.values())
        preprocess_time_per_image = result.speed['preprocess']
        inference_time_per_image = result.speed['inference']
        postprocess_time_per_image = result.speed['postprocess']
        all_time_per_image = preprocess_time_per_image + inference_time_per_image + postprocess_time_per_image
        
        n_l, n_p, n_g, flops = model_info(model.model)
        
        print('-'*20 + '论文上的数据以以下结果为准' + '-'*20)
        print('-'*20 + '论文上的数据以以下结果为准' + '-'*20)
        print('-'*20 + '论文上的数据以以下结果为准' + '-'*20)
        print('-'*20 + '论文上的数据以以下结果为准' + '-'*20)
        print('-'*20 + '论文上的数据以以下结果为准' + '-'*20)

        model_info_table = PrettyTable()
        model_info_table.title = "Model Info"
        model_info_table.field_names = ["GFLOPs", "Parameters", "前处理时间/一张图", "推理时间/一张图", "后处理时间/一张图", "FPS(前处理+模型推理+后处理)", "FPS(推理)", "Model File Size"]
        model_info_table.add_row([f'{flops:.1f}', f'{n_p:,}', 
                                  f'{preprocess_time_per_image / 1000:.6f}s', f'{inference_time_per_image / 1000:.6f}s', 
                                  f'{postprocess_time_per_image / 1000:.6f}s', f'{1000 / all_time_per_image:.2f}', 
                                  f'{1000 / inference_time_per_image:.2f}', f'{get_weight_size(model_path)}MB'])
        print(model_info_table)

        model_metrice_table = PrettyTable()
        model_metrice_table.title = "Model Metrice"
        model_metrice_table.field_names = ["Class Name", "Precision", "Recall", "F1-Score", "mAP50", "mAP75", "mAP50-95"]
        for idx, cls_name in enumerate(model_names):
            model_metrice_table.add_row([
                                        cls_name, 
                                        f"{result.box.p[idx]:.4f}", 
                                        f"{result.box.r[idx]:.4f}", 
                                        f"{result.box.f1[idx]:.4f}", 
                                        f"{result.box.ap50[idx]:.4f}", 
                                        f"{result.box.all_ap[idx, 5]:.4f}", # 50 55 60 65 70 75 80 85 90 95 
                                        f"{result.box.ap[idx]:.4f}"
                                    ])
        model_metrice_table.add_row([
                                    "all(平均数据)", 
                                    f"{result.results_dict['metrics/precision(B)']:.4f}", 
                                    f"{result.results_dict['metrics/recall(B)']:.4f}", 
                                    f"{np.mean(result.box.f1):.4f}", 
                                    f"{result.results_dict['metrics/mAP50(B)']:.4f}", 
                                    f"{np.mean(result.box.all_ap[:, 5]):.4f}", # 50 55 60 65 70 75 80 85 90 95 
                                    f"{result.results_dict['metrics/mAP50-95(B)']:.4f}"
                                ])
        print(model_metrice_table)

        with open(result.save_dir / 'paper_data.txt', 'w+') as f:
            f.write(str(model_info_table))
            f.write('\n')
            f.write(str(model_metrice_table))
        
        print('-'*20, f'结果已保存至{result.save_dir}/paper_data.txt...', '-'*20)
        print('-'*20, f'结果已保存至{result.save_dir}/paper_data.txt...', '-'*20)
        print('-'*20, f'结果已保存至{result.save_dir}/paper_data.txt...', '-'*20)
        print('-'*20, f'结果已保存至{result.save_dir}/paper_data.txt...', '-'*20)
        print('-'*20, f'结果已保存至{result.save_dir}/paper_data.txt...', '-'*20)